In [4]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

In [5]:
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [6]:
# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [13]:
# Start DyNet and defin trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

# Define the model
EMB_SIZE = 64
HID_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE))  # Word embeddings
#fwdLSTM = dy.SimpleRNNBuilder(1, EMB_SIZE, HID_SIZE, model)  # Forward LSTM
#bwdLSTM = dy.SimpleRNNBuilder(1, EMB_SIZE, HID_SIZE, model)  # Backward LSTM
fwdLSTM = dy.VanillaLSTMBuilder(1, EMB_SIZE, HID_SIZE, model)
bwdLSTM = dy.VanillaLSTMBuilder(1, EMB_SIZE, HID_SIZE, model)
W_sm = model.add_parameters((ntags, 2 * HID_SIZE))  # Softmax weights
b_sm = model.add_parameters((ntags))  # Softmax bias

In [14]:
# A function to calculate scores for one value
def calc_scores(words):
    dy.renew_cg()
    word_embs = [dy.lookup(W_emb, x) for x in words]
    fwd_init = fwdLSTM.initial_state()
    fwd_embs = fwd_init.transduce(word_embs)
    bwd_init = bwdLSTM.initial_state()
    bwd_embs = bwd_init.transduce(reversed(word_embs))
    return W_sm * dy.concatenate([fwd_embs[-1], bwd_embs[-1]]) + b_sm


In [15]:
for ITER in range(50):
    # Perform training
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += my_loss.value()
        my_loss.backward()
        trainer.update()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss / len(train), time.time() - start))
    # Perform training
    test_correct = 0.0
    for words, tag in dev:
        scores = calc_scores(words).npvalue()
        predict = np.argmax(scores)
        if predict == tag:
            test_correct += 1
    print("iter %r: test acc=%.4f" % (ITER, test_correct / len(dev)))

iter 0: train loss/sent=1.4733, time=14.80s
iter 0: test acc=0.4131
iter 1: train loss/sent=1.1887, time=14.87s
iter 1: test acc=0.4081
iter 2: train loss/sent=0.9459, time=14.80s
iter 2: test acc=0.4045
iter 3: train loss/sent=0.6957, time=14.36s
iter 3: test acc=0.4127
iter 4: train loss/sent=0.4526, time=14.35s
iter 4: test acc=0.3937
iter 5: train loss/sent=0.2678, time=15.47s
iter 5: test acc=0.4041
iter 6: train loss/sent=0.1552, time=15.31s
iter 6: test acc=0.4041
iter 7: train loss/sent=0.0975, time=15.52s
iter 7: test acc=0.3792
iter 8: train loss/sent=0.0583, time=14.89s
iter 8: test acc=0.3674
iter 9: train loss/sent=0.0359, time=15.15s
iter 9: test acc=0.3765
iter 10: train loss/sent=0.0250, time=14.71s
iter 10: test acc=0.3729
iter 11: train loss/sent=0.0162, time=15.17s
iter 11: test acc=0.3810
iter 12: train loss/sent=0.0120, time=15.82s
iter 12: test acc=0.3805
iter 13: train loss/sent=0.0132, time=15.88s
iter 13: test acc=0.3715
iter 14: train loss/sent=0.0120, time=16

KeyboardInterrupt: 